In [1]:
import rdflib
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
from rdflib import Namespace
from rdflib import URIRef

import csv
from pymantic import sparql

import json
import csv

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\Sofia Baroncini\AppData\Roaming\Python\Python39\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
newg = rdflib.Graph() # we create a new graph
newg.parse("BrunoOderrDIct.ttl", format="ttl")

<Graph identifier=N46687b081e6942018eaeca5ecfacbe4d (<class 'rdflib.graph.Graph'>)>

In [2]:
pang = rdflib.Graph() # we create a new graph
pang.parse("06102022output.ttl", format="ttl")

<Graph identifier=N1bc98e23166a41f6a27c5ed6de03f9ae (<class 'rdflib.graph.Graph'>)>

# Ancillary functions

In [3]:
# ancillary functions


def sparql_query_setting(query, endpoint):# set the endpoint 
  sparql = SPARQLWrapper(endpoint)
  # set the query
  sparql.setQuery(query)
  # set the returned format
  sparql.setReturnFormat(JSON)
  # get the results
  results = sparql.query().convert()
  return results



def wikidata_reconciliation(r, endpoint, q_class=None): # specify the class of which the individual should be instance of
# r = json results of the get request 
  # double check if the entity belongs to the right class
  if 'search' in r and len(r['search']) >= 1:
      if q_class: # if a class is given, it checkes if the wikidata entity belongs to the correct class. 
        query_string = """ASK {<"""+r['search'][0]['concepturi']+"""> wdt:P31 wd:"""+q_class+""". }"""
        res = sparql_query_setting(query_string, endpoint)
        # print("\nRES", query, query_string, res)
        if res["boolean"] == True: # add a timeout

          return [ r['search'][0]['concepturi'] , 'class_match']
        else:
          return [ r['search'][0]['concepturi'] , 'no_class_match']
      else:
        return [ r['search'][0]['concepturi'] , 'no_class_given']
  else:
      return 'not matched'

# having a dictionary containing q terms, query wd for asking its related terms in external vocabulary, as specified by the query.
def alignments_through_wd(dictionary, query, variable_list, endpoint, url_string): 
  all_items = dictionary.items() # ordered dict of tuples
  for item in all_items: # tuple with key-value pairs
    if type(item[1]) == list: 
      for el in item[1]: # item[1]
        if el.startswith("http://www.wikidata.org/entity/"): # in this way, we also filter automatically also the " rec" keys
          print(el, item[0])
          cleaned_el = el.replace("\"","")
          final_query = query.replace("toBeReplaced", cleaned_el)
          res = sparql_query_setting(final_query, endpoint)
          for result in res["results"]["bindings"]:
            for var in variable_list: 
              aligned = url_string+result[var]["value"]
              print(aligned)
              if aligned not in dictionary[item[0]]: 
                dictionary[item[0]].append(aligned)
  return(dictionary)
    
# function already present in the main script - remove/add terms from a dictionary
def remove_add_terms(dictionary, pair_list, instruction_string): # dictionary to be updated, list containing tuples with key-link to be removed
  for pair in pair_list: # pair[0] = name, pair[1] = value
   # print(pair)
    if instruction_string == "remove": 
      if pair[1] in dictionary[pair[0]]:
        dictionary[pair[0]].remove(pair[1])
    elif instruction_string == "add": 
      # if dictionary[pair[0]][0].startswith("http://icondataset.org/") and len(dictionary[pair[0]]) == 1: # we double check that the term is really without an alignment
      if len(dictionary[pair[0]]) <=1:
        dictionary[pair[0]] = [pair[1]]
  return dictionary

# mean of the fuzzywuzzy parameters for word similarity
def fuzz_ratio_mean(name, term):
  ratio1 = fuzz.ratio(name, term)
  ratio2 = fuzz.partial_ratio(name, term)
  ratio3 = fuzz.token_sort_ratio(name, term)
  ratio4 = fuzz.WRatio(name, term)
  mean = (ratio1+ratio2+ratio3+ratio4)/4
  return mean

# open dictionaries stored in json
def open_json(json_file): 
  with open(json_file, mode='r', encoding="utf-8") as jsonfile:
    dictName = json.load(jsonfile)
    return dictName

# save a dictionary in json
def store_in_json(file_name, dictName): 
  with open(file_name, mode='w', encoding="utf-8") as jsonfile:
    json.dump(dictName, jsonfile)


# stop words and others removal. After this passage: ready to reconcile

def rumor_removal(dictionary):
  for item in dictionary.items(): 
    if type(item[1]) == list: 
      for i in range(len(item[1])- 1, -1, -1): # len - 1 = last postition; start -1 = 0; move of -1
        if item[1][i].startswith("http") is False:
          del item[1][i]
  return(dictionary)

# store in csv

def store_csv(file_name, first_line, list_of_lists): 
    with open(file_name, mode='w', newline='', encoding='UTF-8') as my_file:
        file = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        file.writerow(first_line) 
        for l in list_of_lists: 
            file.writerow(l) 
    return(file_name)

# store in csv
# input: list of tuples, where tup[0] is the name and tup[1] is the reconciled link
# store in a json where the first column is the name and the second column is a concat of all the values having that key, with " @ " as separator
def from_tup_to_csv(list_of_tup, file_name, first_line): 
    list_of_lists = []
    tot_dict = {}
    for tup in list_of_tup: 
        if tup[0] not in tot_dict: 
            tot_dict[tup[0]] = []
        tot_dict[tup[0]].append(tup[1])

    for item in tot_dict.items(): 
        list_of_lists.append([item[0], ' @ '.join(item[1])])
        
    with open(file_name, mode='w', newline='') as my_file:
        file = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        file.writerow(first_line) 
        for l in list_of_lists: 
            file.writerow(l) 
    return(file_name) 

import csv

# from dict to csv: 
def store_dict_in_csv(file_name, first_line, dictionary):
    list_of_lists = []
    for tup in dictionary.items(): 
        if tup[0].endswith(" rec"): 
            col1 = tup[0].replace(" rec", "")
            if type(tup[1])== list:
                col2 = ' @ '.join(tup[1])
            else: 
                col2 = tup[1]
            list_of_lists.append([col1, col2])
    final = store_csv(file_name, first_line, list_of_lists)
    return final
    

In [115]:
reality = open_json("matchrc.json")
simulacrum = open_json("matchsimulacrum.json")
panofsky = open_json("symbols_dict.json")

In [127]:
simulacrumnew = open_json("matchsimulacrumnew.json")

In [114]:
reality

{'idolatry': 'https://w3id.org/simulation/data/idolatry',
 'folly': 'https://w3id.org/simulation/data/folly',
 'melancholy': 'https://w3id.org/simulation/data/melancholy',
 'anger': 'https://w3id.org/simulation/data/anger',
 'christ': 'https://w3id.org/simulation/data/christ',
 'unity': 'https://w3id.org/simulation/data/unity',
 'fertility': 'https://w3id.org/simulation/data/fertility',
 'panic': 'https://w3id.org/simulation/data/panic',
 'souls': 'https://w3id.org/simulation/data/souls',
 'envy': 'https://w3id.org/simulation/data/envy',
 'haste': 'https://w3id.org/simulation/data/haste',
 'persuasion': 'https://w3id.org/simulation/data/persuasion',
 'time': 'https://w3id.org/simulation/data/time',
 'devil': 'https://w3id.org/simulation/data/devil',
 'the future': 'https://w3id.org/simulation/data/theFuture',
 'the past': 'https://w3id.org/simulation/data/thePast',
 'luxury': 'https://w3id.org/simulation/data/luxury',
 'god': 'https://w3id.org/simulation/data/god',
 'heaven': 'https://

Cerca quanti match hai nei significati di Panofsky tra i 58 e i 38 simulacri e reality counterpart trovati in comune. Risultato: 29. **NB: da aggiungere alle liste i match trovati con fuzzywuzzy e rifare il calcolo**. Cerca poi quante volte compaiono negli artwork. 

In [119]:
count = 0
matchset = set()
for sim in panofsky.items(): 
    for r in reality: 
        for s in simulacrum: 
            if sim[0] == r: 
                for sy in sim[1]: 
                    if sy == s: 
                        print(sim[0], sy)
                        matchset.add((sim[0], sy))
                        count+=1
print(count)

idolatry apollo
folly bacchus
melancholy bat
anger bear
christ bull
christ ox
souls deer
envy dog
haste dolphin
persuasion dolphin
persuasion mermaid
time dragon biting its tail
luxury fire
egypt hercules
lewdness ixion
pride jupiter
pride sisyphus
love myrtle
purity white
fury lion
discord mars
introspection mirror
everlasting love myrtle
avarice pluto
avarice tantalus
virginity rose
perfidy serpent
death personified skull
greed wolf
29


In [ ]:
# in Panofsky but not in HyperReal: 
idolatry apollo
folly bacchus
anger bear
christ bull
persuasion dolphin
luxury fire
egypt hercules
pride jupiter
pride sisyphus
discord mars
avarice pluto
avarice tantalus





In [122]:
print(matchset)

{('idolatry', 'apollo'), ('persuasion', 'dolphin'), ('purity', 'white'), ('christ', 'bull'), ('lewdness', 'ixion'), ('persuasion', 'mermaid'), ('pride', 'jupiter'), ('anger', 'bear'), ('greed', 'wolf'), ('everlasting love', 'myrtle'), ('envy', 'dog'), ('egypt', 'hercules'), ('love', 'myrtle'), ('melancholy', 'bat'), ('virginity', 'rose'), ('perfidy', 'serpent'), ('fury', 'lion'), ('souls', 'deer'), ('luxury', 'fire'), ('christ', 'ox'), ('introspection', 'mirror'), ('haste', 'dolphin'), ('time', 'dragon biting its tail'), ('folly', 'bacchus'), ('pride', 'sisyphus'), ('death personified', 'skull'), ('avarice', 'tantalus'), ('avarice', 'pluto'), ('discord', 'mars')}


In [121]:
countnotmatch = 0
for r in reality: 
    for s in simulacrum: 
        if (r,s) not in matchset: 
            print(r,s)
            countnotmatch += 1
print(countnotmatch)

idolatry ape
idolatry bacchus
idolatry bat
idolatry bear
idolatry blue
idolatry book
idolatry owl
idolatry serpent
idolatry bull
idolatry ox
idolatry cupid
idolatry deer
idolatry dog
idolatry dolphin
idolatry mermaid
idolatry dove
idolatry dragon biting its tail
idolatry empty throne
idolatry europa
idolatry family
idolatry fire
idolatry fly
idolatry four wings
idolatry hercules
idolatry ibis
idolatry incense burner
idolatry ivy
idolatry ixion
idolatry jewelry
idolatry jupiter
idolatry sisyphus
idolatry myrtle
idolatry white
idolatry lion
idolatry liver
idolatry mars
idolatry masks
idolatry mercury
idolatry mirror
idolatry moses
idolatry mouse
idolatry phaeton
idolatry pillow
idolatry pluto
idolatry tantalus
idolatry rod
idolatry rose
idolatry roses
idolatry satyr
idolatry skull
idolatry snake
idolatry three heads
idolatry three graces
idolatry three
idolatry torch
idolatry victory
idolatry wolf
folly ape
folly apollo
folly bat
folly bear
folly blue
folly book
folly owl
folly serpent
f

the past blue
the past book
the past owl
the past serpent
the past bull
the past ox
the past cupid
the past deer
the past dog
the past dolphin
the past mermaid
the past dove
the past dragon biting its tail
the past empty throne
the past europa
the past family
the past fire
the past fly
the past four wings
the past hercules
the past ibis
the past incense burner
the past ivy
the past ixion
the past jewelry
the past jupiter
the past sisyphus
the past myrtle
the past white
the past lion
the past liver
the past mars
the past masks
the past mercury
the past mirror
the past moses
the past mouse
the past phaeton
the past pillow
the past pluto
the past tantalus
the past rod
the past rose
the past roses
the past satyr
the past skull
the past snake
the past three heads
the past three graces
the past three
the past torch
the past victory
the past wolf
luxury ape
luxury apollo
luxury bacchus
luxury bat
luxury bear
luxury blue
luxury book
luxury owl
luxury serpent
luxury bull
luxury ox
luxury cupid


virginity dog
virginity dolphin
virginity mermaid
virginity dove
virginity dragon biting its tail
virginity empty throne
virginity europa
virginity family
virginity fire
virginity fly
virginity four wings
virginity hercules
virginity ibis
virginity incense burner
virginity ivy
virginity ixion
virginity jewelry
virginity jupiter
virginity sisyphus
virginity myrtle
virginity white
virginity lion
virginity liver
virginity mars
virginity masks
virginity mercury
virginity mirror
virginity moses
virginity mouse
virginity phaeton
virginity pillow
virginity pluto
virginity tantalus
virginity rod
virginity roses
virginity satyr
virginity skull
virginity snake
virginity three heads
virginity three graces
virginity three
virginity torch
virginity victory
virginity wolf
perfidy ape
perfidy apollo
perfidy bacchus
perfidy bat
perfidy bear
perfidy blue
perfidy book
perfidy owl
perfidy bull
perfidy ox
perfidy cupid
perfidy deer
perfidy dog
perfidy dolphin
perfidy mermaid
perfidy dove
perfidy dragon bi

Le coppie che non matchano. prendi tutti i simboli di Panofsky (ogni combinazione key- item della lista), fai tuple, e sottrai dalle tuple match

In [126]:
panofskyset = set()
for item in panofsky.items(): 
    for value in item[1]: 
        panofskyset.add((item[0], value))
# print(panofskyset)
print(len(panofskyset))
differenceset = panofskyset - matchset
print(len(differenceset))

168
139


In [ ]:
Cerca poi quante volte compaiono negli artwork. 

Oderr ha 66 simulation, noi 37 (29+ quelle di fuzzywuzzy), di cui 17 sono in comune con oderr. L'idea è di vedere le 20 che non matchano

In [21]:
rc_not_matched = set()
for rc in panofsky.keys(): 
   # print(rc)
    if rc in reality.keys(): 
        print(rc, "has already a match")
    else:
        rc_not_matched.add(rc)
  #  else: print(rc, "has already a match")
print(rc_not_matched)

idolatry has already a match
folly has already a match
melancholy has already a match
anger has already a match
christ has already a match
unity has already a match
fertility has already a match
panic has already a match
souls has already a match
envy has already a match
haste has already a match
persuasion has already a match
time has already a match
devil has already a match
the future has already a match
the past has already a match
luxury has already a match
god has already a match
heaven has already a match
egypt has already a match
lewdness has already a match
reason has already a match
pride has already a match
love has already a match
purity has already a match
fury has already a match
discord has already a match
introspection has already a match
the tree of life has already a match
everlasting love has already a match
avarice has already a match
salamander has already a match
virginity has already a match
perfidy has already a match
death personified has already a match
pruden

In [23]:
manual_rc_match = {}
for item in rc_not_matched: 
    manual_rc_match[item] = ""

In [ ]:
manual_rc_match["merciless hardships of unmitigated wildness"] = "https://w3id.org/simulation/data/primitiveWildness" # contained in a wider string
manual_rc_match["merciless hardships of unmitigated wildness"] = "https://w3id.org/simulation/data/primitiveWildness" 

Three Graces-three aspects of Venus, kb:threeGraces-theThreeAspectsOfLove # venus: deity of Love

In [25]:
simDict = {}

for r in reality: 
    
    cq1 = '''
    PREFIX kb: <https://w3id.org/simulation/data/> 
    PREFIX ns1: <http://www.w3.org/ns/prov-o-inverses#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX prov: <http://www.w3.org/ns/prov#> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX sim: <https://w3id.org/simulation/ontology/> 
    PREFIX wn: <http://wordnet-rdf.princeton.edu/lemma/> 

    SELECT DISTINCT ?simulacrumLabel ?context WHERE {
    ?sym sim:hasRealityCounterpart ?reality;
        sim:hasSimulacrum ?simulacrum.
    ?reality rdfs:label ?realityLabel.
    ?simulacrum rdfs:label ?simulacrumLabel. 
    optional {?sym sim:hasContext ?context. ?context rdfs:label ?contextLabel}
    FILTER(regex(?realityLabel, "'''+r+'''"))


    }

    '''

    cq1res = newg.query(cq1)

    for row in cq1res:
        simulacrum = row[0]
        if r not in simDict: 
            simDict[r] = [simulacum]
        if len(row) > 1: 
            context = row[1]
            
        print(row)
        

(rdflib.term.URIRef('https://w3id.org/simulation/data/ape-idolatry'), rdflib.term.URIRef('https://w3id.org/simulation/data/ape'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/baal-idolatry'), rdflib.term.URIRef('https://w3id.org/simulation/data/baal'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/babylon-idolatry'), rdflib.term.URIRef('https://w3id.org/simulation/data/babylon'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/dragon-idolatry'), rdflib.term.URIRef('https://w3id.org/simulation/data/dragon'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/egypt-idolatry'), rdflib.term.URIRef('https://w3id.org/simulation/data/egypt'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/goldMetalAndColor-idolatry'), rdflib.term.URIRef('https://w3id.org/simulation/data/goldMetalAndColor'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/greece-idolatry'), rdflib.term.URIRef('https://w3id.org/simulation/data/greece'))
(rdflib.term.URIRef('https://w3id.or

(rdflib.term.URIRef('https://w3id.org/simulation/data/18-danger'), rdflib.term.URIRef('https://w3id.org/simulation/data/18'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/bear-danger'), rdflib.term.URIRef('https://w3id.org/simulation/data/bear'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/brokenRailing-danger'), rdflib.term.URIRef('https://w3id.org/simulation/data/brokenRailing'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/dagger-danger'), rdflib.term.URIRef('https://w3id.org/simulation/data/dagger'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/fire-danger'), rdflib.term.URIRef('https://w3id.org/simulation/data/fire'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/foghorn-danger'), rdflib.term.URIRef('https://w3id.org/simulation/data/foghorn'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/forest-danger'), rdflib.term.URIRef('https://w3id.org/simulation/data/forest'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/gorge-da

(rdflib.term.URIRef('https://w3id.org/simulation/data/bramble-dangerousPride'), rdflib.term.URIRef('https://w3id.org/simulation/data/bramble'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/carolinaRose-loveIsDangerous'), rdflib.term.URIRef('https://w3id.org/simulation/data/carolinaRose'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/chimera-dangerOnLandAndSea'), rdflib.term.URIRef('https://w3id.org/simulation/data/chimera'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/circe-dangerousAttraction'), rdflib.term.URIRef('https://w3id.org/simulation/data/circe'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/coyote-guideOutOfDanger'), rdflib.term.URIRef('https://w3id.org/simulation/data/coyote'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/crab-moneyChangers'), rdflib.term.URIRef('https://w3id.org/simulation/data/crab'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/cresset-oneWhoSignaledInTimeOfDanger'), rdflib.term.URIRef('https://w3id.

(rdflib.term.URIRef('https://w3id.org/simulation/data/undergroundMonster-dangerFromTheSubconscious'), rdflib.term.URIRef('https://w3id.org/simulation/data/undergroundMonster'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/unsanctionedOpeningOfABox-aDangerousActWithUnforeseenConsequences'), rdflib.term.URIRef('https://w3id.org/simulation/data/unsanctionedOpeningOfABox'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/wasp-pettyDanger'), rdflib.term.URIRef('https://w3id.org/simulation/data/wasp'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/wavesOcean-insidiousMoralDangers'), rdflib.term.URIRef('https://w3id.org/simulation/data/wavesOcean'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/sneezing-youKissAStranger'), rdflib.term.URIRef('https://w3id.org/simulation/data/sneezing'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/sneezing-youSneezeForDanger'), rdflib.term.URIRef('https://w3id.org/simulation/data/sneezing'))
(rdflib.term.URIRef('https:/

(rdflib.term.URIRef('https://w3id.org/simulation/data/birdOfPreyGrippingASerpentGod-theDismembermentOfOriginalUnity'), rdflib.term.URIRef('https://w3id.org/simulation/data/birdOfPreyGrippingASerpentGod'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/blue-intelligenceOrderAndUnity'), rdflib.term.URIRef('https://w3id.org/simulation/data/blue'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/red-intelligenceOrderAndUnity'), rdflib.term.URIRef('https://w3id.org/simulation/data/red'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/white-intelligenceOrderAndUnity'), rdflib.term.URIRef('https://w3id.org/simulation/data/white'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/burr-importunity'), rdflib.term.URIRef('https://w3id.org/simulation/data/burr'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/fullerSTeasel-importunity'), rdflib.term.URIRef('https://w3id.org/simulation/data/fullerSTeasel'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/celtic

...

(rdflib.term.URIRef('https://w3id.org/simulation/data/redCatchfly-youthfulLove'), rdflib.term.URIRef('https://w3id.org/simulation/data/redCatchfly'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/redChrysanthemum-iLove'), rdflib.term.URIRef('https://w3id.org/simulation/data/redChrysanthemum'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/redDoublePink-pureAndArdentLove'), rdflib.term.URIRef('https://w3id.org/simulation/data/redDoublePink'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/redRosebud-pureAndLovely'), rdflib.term.URIRef('https://w3id.org/simulation/data/redRosebud'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/roseFlower-loveTriumphant'), rdflib.term.URIRef('https://w3id.org/simulation/data/roseFlower'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/roseFlower-pureLoveWithoutCarnality'), rdflib.term.URIRef('https://w3id.org/simulation/data/roseFlower'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/roseOfSharon-christSLoveF

(rdflib.term.URIRef('https://w3id.org/simulation/data/fish-loveDivinities'), rdflib.term.URIRef('https://w3id.org/simulation/data/fish'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/peony-feminineLoveliness'), rdflib.term.URIRef('https://w3id.org/simulation/data/peony'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/alabaster-purity'), rdflib.term.URIRef('https://w3id.org/simulation/data/alabaster'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/alb-purity'), rdflib.term.URIRef('https://w3id.org/simulation/data/alb'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/angelHoldingALily-purity'), rdflib.term.URIRef('https://w3id.org/simulation/data/angelHoldingALily'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/asperges-purity'), rdflib.term.URIRef('https://w3id.org/simulation/data/asperges'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/basin-purity'), rdflib.term.URIRef('https://w3id.org/simulation/data/basin'))
(rdflib.term.URIRef('htt

(rdflib.term.URIRef('https://w3id.org/simulation/data/amice-purityOfHeartAndSoul'), rdflib.term.URIRef('https://w3id.org/simulation/data/amice'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/apron-purityOfLifeAndConduct'), rdflib.term.URIRef('https://w3id.org/simulation/data/apron'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/baptismByFire-restorationOfPrimordialPurity'), rdflib.term.URIRef('https://w3id.org/simulation/data/baptismByFire'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/blackAndWhite-humilityAndPurityOfLifeRespectively'), rdflib.term.URIRef('https://w3id.org/simulation/data/blackAndWhite'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/blossom-yourPurityEqualsYourLoveliness'), rdflib.term.URIRef('https://w3id.org/simulation/data/blossom'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/convertingBaseMetalIntoGold-regainingThePrimordialPurityOfHumanNature'), rdflib.term.URIRef('https://w3id.org/simulation/data/convertingBaseMeta

(rdflib.term.URIRef('https://w3id.org/simulation/data/scorpion-mischiefAndDiscord'), rdflib.term.URIRef('https://w3id.org/simulation/data/scorpion'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/b-introspection'), rdflib.term.URIRef('https://w3id.org/simulation/data/b'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/fishing-introspection'), rdflib.term.URIRef('https://w3id.org/simulation/data/fishing'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/g-introspection'), rdflib.term.URIRef('https://w3id.org/simulation/data/g'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/hamlet-introspection'), rdflib.term.URIRef('https://w3id.org/simulation/data/hamlet'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/highPriestessTarot-introspection'), rdflib.term.URIRef('https://w3id.org/simulation/data/highPriestessTarot'))
(rdflib.term.URIRef('https://w3id.org/simulation/data/judgmentTarot-introspection'), rdflib.term.URIRef('https://w3id.org/simulation/data/j

KeyboardInterrupt: 

In [59]:
#simulacrumUri = []
RcUri = []

with open("BrunocommonRcWithRespectiveSimulacrum.csv", mode='r', encoding="utf-8") as csvfile:
    commonDict = csv.DictReader(csvfile)
    for row in commonDict:
        # print(row["rc"])
        RcUri.append(row["rc"])

https://w3id.org/simulation/data/anger
https://w3id.org/simulation/data/avarice
https://w3id.org/simulation/data/christ
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/greed
https://w3id.org/simulation/data/greed
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/folly
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data

https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/egypt
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/discord
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/envy
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simulation/data/fertility
https://w3id.org/simul

In [57]:
print(labelRc)

{}


In [60]:
#labelSimulacrum = {}
labelRc = {}
for uri in RcUri: 
    cq1 = '''
    PREFIX kb: <https://w3id.org/simulation/data/> 
    PREFIX ns1: <http://www.w3.org/ns/prov-o-inverses#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX prov: <http://www.w3.org/ns/prov#> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX sim: <https://w3id.org/simulation/ontology/> 
    PREFIX wn: <http://wordnet-rdf.princeton.edu/lemma/> 

    SELECT DISTINCT ?label WHERE {
    <'''+uri+'''> rdfs:label ?label.

    }

    '''

    cq1res = newg.query(cq1)

    for row in cq1res:
        rc = str(row[0])
        labelRc[uri] = rc
            
print(labelRc)

{'https://w3id.org/simulation/data/anger': 'anger', 'https://w3id.org/simulation/data/avarice': 'avarice', 'https://w3id.org/simulation/data/christ': 'Christ', 'https://w3id.org/simulation/data/fertility': 'fertility', 'https://w3id.org/simulation/data/greed': 'greed', 'https://w3id.org/simulation/data/folly': 'folly', 'https://w3id.org/simulation/data/everlastingLove': 'everlasting love', 'https://w3id.org/simulation/data/fury': 'fury', 'https://w3id.org/simulation/data/god': 'God', 'https://w3id.org/simulation/data/heaven': 'heaven', 'https://w3id.org/simulation/data/haste': 'haste', 'https://w3id.org/simulation/data/egypt': 'Egypt', 'https://w3id.org/simulation/data/envy': 'envy', 'https://w3id.org/simulation/data/deathPersonified': 'death (Personified)', 'https://w3id.org/simulation/data/discord': 'discord', 'https://w3id.org/simulation/data/luxury': 'luxury', 'https://w3id.org/simulation/data/melancholy': 'melancholy', 'https://w3id.org/simulation/data/love': 'love', 'https://w3id

In [86]:
SimDict = {}

with open("BrunocommonRcWithRespectiveSimulacrum.csv", mode='r', encoding="utf-8") as csvfile:
    commonDict = csv.DictReader(csvfile)
    for row in commonDict:
        
        rc = row["rc"]
        sim = row["simulacrum"]
        rcname = labelRc[rc]
        simname = labelSimulacrum[sim]
        if rcname not in SimDict: 
            SimDict[rcname] = []
        if simname not in SimDict[rcname]:
            SimDict[rcname].append(simname)
        # print(row["rc"])
       # RcUri.append(row["rc"])
print(SimDict)

{'anger': ['wolf', 'whin', 'Ares/Mars', 'ember', 'flaming torch', 'furze', 'in swarms', 'purple-red', 'serpent at the center of the Round of Existence', 'fever', 'fire', 'fist', 'peony', 'petunia', 'red', 'rending', 'rending garments', 'boar', 'rooster', 'Satan', 'tiger', 'volcano', 'water element'], 'avarice': ['wolf', 'blindfold', 'coins', 'dark yellow', 'Jew', 'rake (implement)', 'scarlet auricula', 'frog', 'goat', 'hyena', 'itchy palm', 'leech', 'Mammon', 'mole (animal)', 'monkey', 'owl', 'purse', 'spider', 'squirrel', 'toad', 'weasel', 'whale'], 'Christ': ['X', 'Agnus Dei', 'book with “alpha” and “omega” printed on it', '37', 'alpha and omega', 'anchor', 'balances', 'beacon', 'caladrius', 'carpenter', 'cedar', 'closed book with seven seals and lamb', 'Constantine’s labarum', 'cornerstone', 'crane (bird)', 'daisy', 'day star', 'dew', 'diamond (gem)', 'door', 'eagle', 'east', 'grape leaves or vine', 'lamb with cross', 'lamb with nimbus', 'older brother', 'purple dove', 'Rod of Jesse

In [87]:
store = store_in_json("simDictLabel.json", SimDict)

In [65]:
panofsky

{'worldliness, insincerity, falsehood': ['a pair of masks, one of a young woman, the other of a old malevolent man'],
 'angel glad that the soul of the pope had been received among the blessed spirits': ['angel smiling while carrying the coffin of the pope'],
 'angel grieved that the world had been deprived of such a man': ['angel wepting while carrying the coffin of the pope'],
 'everything subhuman in man (lust, greed, gluttony, shamelessness)': ['ape'],
 'lower soul': ['ape', 'venus, cupid, vulcan'],
 'idolatry': ['apollo'],
 'tools of a force beyond their will and consciousness': ['arciers without arrows'],
 'folly': ['bacchus'],
 'melancholy': ['bat'],
 'anger': ['bear'],
 'affection changes with luck': ['blind fortune blindfolding cupid and putting him on a sphere'],
 'generating panic terror': ['blowing in a cone shell'],
 'fertilising': ['blowing in a tube'],
 'hue of heaven': ['blue'],
 'attribute of wisdom': ['book', 'owl', 'serpent'],
 'antidotes against sexual passion (marr

In [88]:
for tup in panofsky.items():
        for item in SimDict.items(): 
            if tup[0] == item[0]: 
                for t in tup[1]: 
                   # print("panofsky simulacrum", t, "for the rc", tup[0])
                    for i in item[1]:
                     #   print("oderr simulacrum", i, "for the rc", item[0])
                        mean = fuzz_ratio_mean(t, i)
                        if mean > 70: 
                           # print("panofsky simulacrum", t, "for the rc", tup[0])
                           # print("oderr simulacrum", i, "for the rc", item[0])
                            print("matched simulacra:", t, " - ", i, "for the rc", tup[0], " - ", item[0])

matched simulacra: bear  -  ember for the rc anger  -  anger
matched simulacra: bear  -  boar for the rc anger  -  anger
matched simulacra: cone shell  -  shell for the rc fertility  -  fertility
matched simulacra: dog  -  dog for the rc envy  -  envy
matched simulacra: dragon biting its tail  -  dragon biting its own tail for the rc time  -  time
matched simulacra: myrtle  -  myrtle for the rc love  -  love
matched simulacra: putti with dolphins  -  pillar with a dolphin for the rc love  -  love
matched simulacra: putti with dolphins  -  dolphin for the rc love  -  love
matched simulacra: white  -  white dress for the rc purity  -  purity
matched simulacra: white  -  white for the rc purity  -  purity
matched simulacra: white  -  white bird for the rc purity  -  purity
matched simulacra: white  -  white dove for the rc purity  -  purity
matched simulacra: white  -  white flower for the rc purity  -  purity
matched simulacra: white  -  white knight for the rc purity  -  purity
matched 

In [96]:
#simulacrumUri = []
rc2Uri = []
sim2Uri = []
with open("commonsimulacrumWithRespectiveRc.csv", mode='r', encoding="utf-8") as csvfile:
    commonDict = csv.DictReader(csvfile)
    for row in commonDict:
        # print(row["rc"])
        rc2Uri.append(row["rc"])
        sim2Uri.append(row["simulacrum"])

In [97]:
#labelSimulacrum = {}
label2Rc = {}
for uri in rc2Uri: 
    cq1 = '''
    PREFIX kb: <https://w3id.org/simulation/data/> 
    PREFIX ns1: <http://www.w3.org/ns/prov-o-inverses#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX prov: <http://www.w3.org/ns/prov#> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX sim: <https://w3id.org/simulation/ontology/> 
    PREFIX wn: <http://wordnet-rdf.princeton.edu/lemma/> 

    SELECT DISTINCT ?label WHERE {
    <'''+uri+'''> rdfs:label ?label.

    }

    '''

    cq1res = newg.query(cq1)

    for row in cq1res:
        rc = str(row[0])
        label2Rc[uri] = rc
            
print(label2Rc)

{'https://w3id.org/simulation/data/4thWeddingAnniversaries': '4th wedding anniversaries', 'https://w3id.org/simulation/data/akashicRecords': 'akashic records', 'https://w3id.org/simulation/data/andOtherSaintsFamousForTheirLearning': 'and other saints famous for their learning', 'https://w3id.org/simulation/data/theStarryVault': 'the Starry Vault', 'https://w3id.org/simulation/data/theUnveilingOfTruth': 'the unveiling of truth', 'https://w3id.org/simulation/data/theWhigParty': 'the Whig party', 'https://w3id.org/simulation/data/theWisdomOfTheDharmaDhatu': 'the wisdom of the Dharma-Dhatu', 'https://w3id.org/simulation/data/timeAndSpace': 'time and space', 'https://w3id.org/simulation/data/witchesAbout': 'witches about', 'https://w3id.org/simulation/data/authors': 'authors', 'https://w3id.org/simulation/data/cyprian': 'Cyprian', 'https://w3id.org/simulation/data/secretOrDivineKnowledge': 'secret or divine knowledge', 'https://w3id.org/simulation/data/spiritualOrMysticalPower': 'spiritual 

In [98]:
#labelSimulacrum = {}
label2Sim = {}
for uri in sim2Uri: 
    cq1 = '''
    PREFIX kb: <https://w3id.org/simulation/data/> 
    PREFIX ns1: <http://www.w3.org/ns/prov-o-inverses#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX prov: <http://www.w3.org/ns/prov#> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX sim: <https://w3id.org/simulation/ontology/> 
    PREFIX wn: <http://wordnet-rdf.princeton.edu/lemma/> 

    SELECT DISTINCT ?label WHERE {
    <'''+uri+'''> rdfs:label ?label.

    }

    '''

    cq1res = newg.query(cq1)

    for row in cq1res:
        sim = str(row[0])
        label2Sim[uri] = sim
            
print(label2Sim)

{'https://w3id.org/simulation/data/book': 'book', 'https://w3id.org/simulation/data/blue': 'blue', 'https://w3id.org/simulation/data/bull': 'bull', 'https://w3id.org/simulation/data/bear': 'bear', 'https://w3id.org/simulation/data/threeGraces': 'Three Graces', 'https://w3id.org/simulation/data/dove': 'dove', 'https://w3id.org/simulation/data/fire': 'fire', 'https://w3id.org/simulation/data/ibis': 'ibis', 'https://w3id.org/simulation/data/lion': 'lion', 'https://w3id.org/simulation/data/dog': 'dog', 'https://w3id.org/simulation/data/myrtle': 'myrtle', 'https://w3id.org/simulation/data/ox': 'ox', 'https://w3id.org/simulation/data/mirror': 'mirror', 'https://w3id.org/simulation/data/masks': 'masks', 'https://w3id.org/simulation/data/skull': 'skull', 'https://w3id.org/simulation/data/serpent': 'serpent', 'https://w3id.org/simulation/data/wolf': 'wolf', 'https://w3id.org/simulation/data/ape': 'ape', 'https://w3id.org/simulation/data/owl': 'owl', 'https://w3id.org/simulation/data/mouse': 'mo

In [102]:

Simulacrum2Dict = {}
with open("commonsimulacrumWithRespectiveRc.csv", mode='r', encoding="utf-8") as csvfile:
    commonDict = csv.DictReader(csvfile)
    for row in commonDict:
        
        rc = row["rc"]
        sim = row["simulacrum"]
        rcname = label2Rc[rc]
        simname = label2Sim[sim]
        if simname not in Simulacrum2Dict: 
            Simulacrum2Dict[simname] = []
        if rcname not in Simulacrum2Dict[simname]:
            Simulacrum2Dict[simname].append(rcname)
        # print(row["rc"])
       # RcUri.append(row["rc"])
print(Simulacrum2Dict)

{'book': ['4th wedding anniversaries', 'akashic records', 'and other saints famous for their learning', 'authors', 'Cyprian', 'secret or divine knowledge', 'spiritual or mystical power', 'the Creation', 'the Doctors of the Church', 'the totality of God’s decrees', 'the Virtues', 'Adelante Fraternity', 'Augustine', 'Bernard of Clairvaux', 'Bonaventure', 'Clio', 'knowledge', 'magic', 'mercy', 'prophets', 'scholarship', 'SS Anne', 'teaching', 'the Apostles', 'the archangel Uriel', 'the Koran', 'the sibyls', 'the universe', 'Thomas Aquinas', 'truth', 'virtues', 'wisdom', 'the Bible', 'Fukurokujiu'], 'blue': ['the Starry Vault', 'the unveiling of truth', 'the Whig party', 'the wisdom of the Dharma-Dhatu', 'time and space', 'witches about', 'Akshobhya', 'Aphrodite/Venus)', 'beauty', 'calmness', 'clouds', 'coldness', 'contemplation', 'courage', 'despondency', 'devotion', 'east', 'expiation', 'fairness', 'faithfulness', 'fidelity', 'folly', 'foundation', 'glory', 'half-mourning', 'harmony', 'H

In [104]:
for tup in panofsky.items():
        for item in Simulacrum2Dict.items(): 
            for t in tup[1]:
                if t == item[0]: 
                   # print("panofsky simulacrum", t, "for the rc", tup[0])
                    for i in item[1]:
                     #   print("oderr simulacrum", i, "for the rc", item[0])
                        mean = fuzz_ratio_mean(tup[0], i)
                        if mean > 70: 
                           # print("panofsky simulacrum", t, "for the rc", tup[0])
                           # print("oderr simulacrum", i, "for the rc", item[0])
                            print("matched reality counterpart:", tup[0], " - ", i, "- for the simulacrum:", t, " - ", item[0])

matched reality counterpart: anger  -  danger - for the simulacrum: bear  -  bear
matched reality counterpart: hue of heaven  -  the heavens - for the simulacrum: blue  -  blue
matched reality counterpart: hue of heaven  -  heaven - for the simulacrum: blue  -  blue
matched reality counterpart: attribute of wisdom  -  wisdom - for the simulacrum: book  -  book
matched reality counterpart: attribute of wisdom  -  wisdom - for the simulacrum: serpent  -  serpent
matched reality counterpart: attribute of wisdom  -  wisdom - for the simulacrum: owl  -  owl
matched reality counterpart: christ  -  Christ - for the simulacrum: ox  -  ox
matched reality counterpart: souls  -  the soul - for the simulacrum: deer  -  deer
matched reality counterpart: envy  -  envy - for the simulacrum: dog  -  dog
matched reality counterpart: holy spirit  -  the Holy Spirit - for the simulacrum: dove  -  dove
matched reality counterpart: four seasons  -  the four seasons - for the simulacrum: four wings  -  four

# Subgraph of Panofsky dataset about symbols

tutorial to use construct: https://euclid-project.eu/modules/chapter2.html#:~:text=Queries%20using%20CONSTRUCT 

what we want to know:
* all the metadata of the artworks having at least a symbolic meaning: title, title label, etc.
* the interpretation recognising the symbol
* the simulation detail
* detail of the source of the simulation (all the relation from the node? e.g. books, books content) 
* detail of the interpretation



In [13]:
# exercises on construct
 

cq1 = '''
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>


CONSTRUCT{ ?image icon:hasSymbol ?symbol.
?icrec icon:recognizedImage ?image; 
    icon:aboutWorkOfArt ?artwork;
    crm:P14_carried_out_by ?person. }
where {  
VALUES ?artwork {data:ART1043 data:ART1044}
?s icon:aboutWorkOfArt  ?artwork.
?image icon:hasSymbol ?symbol; 
    ^icon:recognizedImage ?icrec.
?icrec icon:aboutWorkOfArt ?artwork; 
    crm:P14_carried_out_by ?person. 
?artwork ^crm:P65_shows_visual_item ?obj. 
?obj ?rel ?o. 
    
OPTIONAL {?icrec cito:citesAsSource ?source}
OPTIONAL {?icrec cito:citesAsEvidence ?Evidence}

}
# WHERE {    data:ART1001 <>|!<>* ?o. }

'''

cq1res = pang.query(cq1)

for row in cq1res:
    print(row)
        

(rdflib.term.URIRef('http://icondataset.org/ART1043-ICREC5'), rdflib.term.URIRef('https://w3id.org/icon/ontology/recognizedImage'), rdflib.term.URIRef('http://icondataset.org/ART1043-IMG5'))
(rdflib.term.URIRef('http://icondataset.org/ART1044-ICREC4'), rdflib.term.URIRef('https://w3id.org/icon/ontology/aboutWorkOfArt'), rdflib.term.URIRef('http://icondataset.org/ART1044'))
(rdflib.term.URIRef('http://icondataset.org/ART1044-ICREC4'), rdflib.term.URIRef('https://w3id.org/icon/ontology/recognizedImage'), rdflib.term.URIRef('http://icondataset.org/ART1044-IMG4'))
(rdflib.term.URIRef('http://icondataset.org/ART1044-IMG3'), rdflib.term.URIRef('https://w3id.org/icon/ontology/hasSymbol'), rdflib.term.URIRef('http://icondataset.org/the-grace-turning-his-back-favor-conferred'))
(rdflib.term.URIRef('http://icondataset.org/ART1044-ICREC3'), rdflib.term.URIRef('https://w3id.org/icon/ontology/aboutWorkOfArt'), rdflib.term.URIRef('http://icondataset.org/ART1044'))
(rdflib.term.URIRef('http://icondat

In [ ]:
# exercises on construct
 

cq1 = '''
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>


SELECT *
where {  
VALUES ?artwork {data:ART1001test data:ART1002test data:ART1003test data:ART1004 data:ART1008test data:ART1009test data:ART1011test data:ART1013test data:ART1017test data:ART1043 data:ART1044 data:ART1066 data:ART1104 data:ART1124 data:ART1126 data:ART1127 data:ART1128 data:ART1129 data:ART1132 data:ART1158}


?icrec icon:aboutWorkOfArt ?artwork; 
    crm:P14_carried_out_by ?person; 
    icon:recognizedImage ?image. 
#?obj crm:P65_shows_visual_item ?artwork. 
#?production crm:P108_has_produced ?obj.

    
# OPTIONAL {?icrec cito:citesAsSource ?source}
# OPTIONAL {?icrec cito:citesAsEvidence ?Evidence}
# OPTIONAL {?production crm:P4_has_time_span ?time}
# OPTIONAL {?obj crm:P53_has_former_or_current_location ?location. 
#            ?location rdfs:label ?placeLabel}
# OPTIONAL {?obj crm:P2_has_type ?type}
#OPTIONAL {?obj crm:P102_has_title ?title. ?title rdfs:label ?label.}
}
# WHERE {    data:ART1001 <>|!<>* ?o. }

'''

cq1res = pang.query(cq1)

for row in cq1res:
    print(row)
        

In [ ]:
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>
        
        
CONSTRUCT { * }
WHERE {  dbr:List_of_Egyptian_deities dbo:wikiPageWikiLink  ?god .
?god dct:subject ?deity .
{?deity skos:broader dbc:Egyptian_deities} UNION {?deity skos:broader dbc:Egyptian_gods}
OPTIONAL { ?god foaf:depiction ?depiction  }
OPTIONAL { ?god dbp:parents ?parent }
OPTIONAL {?god dbp:symbol ?symbol }
OPTIONAL {?god dbp:consort ?consort }
OPTIONAL {?god dbp:cultCenter ?cultcenter }
OPTIONAL {?god dbp:godOf ?godof }
OPTIONAL {?god dbp:offspring ?offspring }
OPTIONAL {?god dbp:sibilings ?sibilings} }

# Panofsky Artworks

In [135]:
# all the artworks having a symbol assocuated

cq1 = '''
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT * WHERE {
  ?s ?p ?o.
}LIMIT 10

'''

cq1res = pang.query(cq1)

for row in cq1res:
    print(row)


(rdflib.term.URIRef('http://icondataset.org/ART1122-AM11'), rdflib.term.URIRef('https://w3id.org/icon/ontology/hasRecAttribute'), rdflib.term.URIRef('http://icondataset.org/ART1122-IMG2'))
(rdflib.term.URIRef('http://icondataset.org/ART1003test-ICDESC'), rdflib.term.URIRef('https://w3id.org/icon/ontology/isListedIn'), rdflib.term.URIRef('http://icondataset.org/ART1003test-ICREC3'))
(rdflib.term.URIRef('http://www.worldcat.org/oclc/265570808'), rdflib.term.URIRef('http://purl.org/spar/cito/citesForInformation'), rdflib.term.URIRef('http://icondataset.org/ART1066-PREICREC12'))
(rdflib.term.URIRef('https://w3id.org/icon/ontology/PreiconographicalDescription'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://icondataset.org/ART1268-PREICDESC'))
(rdflib.term.Literal('1835-12-31T23:59:59'), rdflib.term.URIRef('http://www.cidoc-crm.org/cidoc-crm/P82b_end_of_the_end'), rdflib.term.URIRef('http://icondataset.org/1835'))
(rdflib.term.URIRef('http

In [188]:
# all the artworks having a symbol associated
artworkSet = set()
cq1 = '''
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?artwork WHERE {
  ?image icon:hasSymbol ?symbol; 
      ^icon:recognizedImage / icon:aboutWorkOfArt ?artwork .

}

'''

cq1res = pang.query(cq1)

for row in cq1res:
    artworkSet.add(str(row[0]))
    print(row)


(rdflib.term.URIRef('http://icondataset.org/ART1001test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1002test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1003test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1004'),)
(rdflib.term.URIRef('http://icondataset.org/ART1008test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1009test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1011test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1013test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1017test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1043'),)
(rdflib.term.URIRef('http://icondataset.org/ART1044'),)
(rdflib.term.URIRef('http://icondataset.org/ART1066'),)
(rdflib.term.URIRef('http://icondataset.org/ART1104'),)
(rdflib.term.URIRef('http://icondataset.org/ART1124'),)
(rdflib.term.URIRef('http://icondataset.org/ART1126'),)
(rdflib.term.URIRef('http://icondataset.org/ART1127'),)
(rdflib.term.URIRef('http://icondataset.org/ART1128'),)
(rdflib.term.URI

In [189]:
print(len(artworkSet))

83


In [190]:
artworkSet

{'http://icondataset.org/ART1001test',
 'http://icondataset.org/ART1002test',
 'http://icondataset.org/ART1003test',
 'http://icondataset.org/ART1004',
 'http://icondataset.org/ART1008test',
 'http://icondataset.org/ART1009test',
 'http://icondataset.org/ART1011test',
 'http://icondataset.org/ART1013test',
 'http://icondataset.org/ART1017test',
 'http://icondataset.org/ART1043',
 'http://icondataset.org/ART1044',
 'http://icondataset.org/ART1066',
 'http://icondataset.org/ART1104',
 'http://icondataset.org/ART1124',
 'http://icondataset.org/ART1126',
 'http://icondataset.org/ART1127',
 'http://icondataset.org/ART1128',
 'http://icondataset.org/ART1129',
 'http://icondataset.org/ART1132',
 'http://icondataset.org/ART1158',
 'http://icondataset.org/ART1196',
 'http://icondataset.org/ART1197',
 'http://icondataset.org/ART1198',
 'http://icondataset.org/ART1199',
 'http://icondataset.org/ART1200',
 'http://icondataset.org/ART1201',
 'http://icondataset.org/ART1202',
 'http://icondataset.or

In [139]:
# count of how many times the symbols appear in the artwork

cq1 = '''
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?symbol (count(?artwork) as ?tot) WHERE {
  ?image icon:hasSymbol ?symbol; 
      ^icon:recognizedImage / icon:aboutWorkOfArt ?artwork .

} GROUP BY ?symbol ?tot
ORDER BY DESC(?tot)

'''

cq1res = pang.query(cq1)

for row in cq1res:
    print(row[0], row[1])

http://icondataset.org/fall-of-phaeton-fate-of-every-temerarius 4
http://icondataset.org/three-headed-companion-of-serapis-three-forms-of-time-past-present-future 4
http://icondataset.org/dragon-biting-its-tail-time 3
http://icondataset.org/phaeton-the-presumptuous 3
http://icondataset.org/three-headed-companion-of-serapis-prudence 3
http://icondataset.org/satyr-egyptian-desert-wilderness 2
http://icondataset.org/cupid-amor-carnalis 2
http://icondataset.org/four-wings-four-seasons 2
http://icondataset.org/slave-human-soul-in-so-far-as-it-is-devoid-of-freedom 2
http://icondataset.org/fettered-ape-subjugation-of-base-emotions 2
http://icondataset.org/empty-throne-invisible-presence-of-an-immortal 2
http://icondataset.org/highest-zone-of-the-tomb-supercelestial-sphere 2
http://icondataset.org/lowest-zone-of-the-tomb-realm-of-sheer-matter 2
http://icondataset.org/central-zone-of-the-tomb-realm-of-nature-the-only-sphere-subject-to-time 2
http://icondataset.org/four-times-of-day-four-humours

In [169]:
commonSimulation = ['deer-souls', 'dove-holy-spirit', 'white-purity', 'cone-shell-fertility', 'bat-melancholy', 'dragon-biting-its-tail-time', 'blue-hue-of-heaven', 'dog-envy', 'skull-death-personified', 'dolphin-haste', 'four-wings-four-seasons', 'three-heads-three-forms-of-time-past-present-future', 'ox-christ', 'serpent-perfidy', 'myrtle-everlasting-love', 'ixion-lewdness', 'myrtle-love', 'lion-fury', 'wolf-greed', 'mermaid-persuasion', 'mirror-introspection', 'rose-virginity', 'ivy-eternal-love']
onlyPanofskySimulation = ["tantalus-avarice", "apollo-idolatry","bacchus-folly","bear-anger","bull-christ","dolphin-persuasion","fire-luxury","hercules-egypt","jupiter-pride","mars-discord","pluto-avarice","sisyphus-pride"]
totPanofskySimulation = commonSimulation + onlyPanofskySimulation

In [180]:
totPanofskySimulation = commonSimulation + onlyPanofskySimulation
print(totPanofskySimulation)


['deer-souls', 'dove-holy-spirit', 'white-purity', 'cone-shell-fertility', 'bat-melancholy', 'dragon-biting-its-tail-time', 'blue-hue-of-heaven', 'dog-envy', 'skull-death-personified', 'dolphin-haste', 'four-wings-four-seasons', 'three-heads-three-forms-of-time-past-present-future', 'ox-christ', 'serpent-perfidy', 'myrtle-everlasting-love', 'ixion-lewdness', 'myrtle-love', 'lion-fury', 'wolf-greed', 'mermaid-persuasion', 'mirror-introspection', 'rose-virginity', 'ivy-eternal-love', 'tantalus-avarice', 'apollo-idolatry', 'bacchus-folly', 'bear-anger', 'bull-christ', 'dolphin-persuasion', 'fire-luxury', 'hercules-egypt', 'jupiter-pride', 'mars-discord', 'pluto-avarice', 'sisyphus-pride']


In [151]:
count = 0
pansymb = set()
for row in cq1res: 
    final = str(row[0]).replace("http://icondataset.org/","")
    pansymb.add(final)
for sim in commonSimulation: 
    if sim in pansymb: 
        count+=1
    else: 
        print(sim, "not in Panofsky data")
print(count)


30


In [161]:
count = 0
for sim in onlyPanofskySimulation: 
    if sim in pansymb: 
        count+=1
    else: 
        print(sim, "not in Panofsky data")
print(count)
print(len(onlyPanofskySimulation))

12
12


In [179]:
artworkSim = set()
for sim in commonSimulation: 
   # all the artworks having a symbol associated
    cq2 = '''
    prefix data: <http://icondataset.org/> 
    PREFIX icon: <https://w3id.org/icon/ontology/> 
    prefix sim: <https://w3id.org/simulation/ontology/>
    prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
    prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

    SELECT DISTINCT ?artwork WHERE {
      ?image icon:hasSymbol <http://icondataset.org/'''+sim+'''>; 
          ^icon:recognizedImage / icon:aboutWorkOfArt ?artwork .
    }

    '''

    cq2res = pang.query(cq2)

    for row in cq2res:
        artworkSim.add(str(row[0]))
        print(row)


(rdflib.term.URIRef('http://icondataset.org/ART1004'),)
(rdflib.term.URIRef('http://icondataset.org/ART1009test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1017test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1002test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1374'),)
(rdflib.term.URIRef('http://icondataset.org/ART1043'),)
(rdflib.term.URIRef('http://icondataset.org/ART1266'),)
(rdflib.term.URIRef('http://icondataset.org/ART1274'),)
(rdflib.term.URIRef('http://icondataset.org/ART1017test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1283'),)
(rdflib.term.URIRef('http://icondataset.org/ART1467'),)
(rdflib.term.URIRef('http://icondataset.org/ART1422'),)
(rdflib.term.URIRef('http://icondataset.org/ART1424'),)
(rdflib.term.URIRef('http://icondataset.org/ART1271'),)
(rdflib.term.URIRef('http://icondataset.org/ART1273'),)
(rdflib.term.URIRef('http://icondataset.org/ART1271'),)
(rdflib.term.URIRef('http://icondataset.org/ART1199'),)
(rdflib.term.URIRef('http://icon

In [187]:
artworkSim

{'http://icondataset.org/ART1001test',
 'http://icondataset.org/ART1002test',
 'http://icondataset.org/ART1004',
 'http://icondataset.org/ART1009test',
 'http://icondataset.org/ART1011test',
 'http://icondataset.org/ART1017test',
 'http://icondataset.org/ART1043',
 'http://icondataset.org/ART1124',
 'http://icondataset.org/ART1199',
 'http://icondataset.org/ART1266',
 'http://icondataset.org/ART1271',
 'http://icondataset.org/ART1273',
 'http://icondataset.org/ART1274',
 'http://icondataset.org/ART1283',
 'http://icondataset.org/ART1288',
 'http://icondataset.org/ART1338',
 'http://icondataset.org/ART1374',
 'http://icondataset.org/ART1422',
 'http://icondataset.org/ART1424',
 'http://icondataset.org/ART1467'}

In [184]:
#totPanofskySimulation

artTotPanSim = set() # the total of artworks having a symbol from the total list we considered of Panofsky
for sim in totPanofskySimulation: 
   # all the artworks having a symbol assocuated
    cq2 = '''
    prefix data: <http://icondataset.org/> 
    PREFIX icon: <https://w3id.org/icon/ontology/> 
    prefix sim: <https://w3id.org/simulation/ontology/>
    prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
    prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

    SELECT DISTINCT ?artwork WHERE {
      ?image icon:hasSymbol <http://icondataset.org/'''+sim+'''>; 
          ^icon:recognizedImage / icon:aboutWorkOfArt ?artwork .
    }

    '''

    cq2res = pang.query(cq2)

    for row in cq2res:
        artTotPanSim.add(str(row[0]))
        print(row)
print(len(artTotPanSim))

(rdflib.term.URIRef('http://icondataset.org/ART1004'),)
(rdflib.term.URIRef('http://icondataset.org/ART1009test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1017test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1002test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1374'),)
(rdflib.term.URIRef('http://icondataset.org/ART1043'),)
(rdflib.term.URIRef('http://icondataset.org/ART1266'),)
(rdflib.term.URIRef('http://icondataset.org/ART1274'),)
(rdflib.term.URIRef('http://icondataset.org/ART1017test'),)
(rdflib.term.URIRef('http://icondataset.org/ART1283'),)
(rdflib.term.URIRef('http://icondataset.org/ART1467'),)
(rdflib.term.URIRef('http://icondataset.org/ART1422'),)
(rdflib.term.URIRef('http://icondataset.org/ART1424'),)
(rdflib.term.URIRef('http://icondataset.org/ART1271'),)
(rdflib.term.URIRef('http://icondataset.org/ART1273'),)
(rdflib.term.URIRef('http://icondataset.org/ART1271'),)
(rdflib.term.URIRef('http://icondataset.org/ART1199'),)
(rdflib.term.URIRef('http://icon

In [191]:
print(artTotPanSim - artworkSim)

{'http://icondataset.org/ART1525', 'http://icondataset.org/ART1013test', 'http://icondataset.org/ART1066', 'http://icondataset.org/ART1126'}


In [186]:
artTotPanSim

{'http://icondataset.org/ART1001test',
 'http://icondataset.org/ART1002test',
 'http://icondataset.org/ART1004',
 'http://icondataset.org/ART1009test',
 'http://icondataset.org/ART1011test',
 'http://icondataset.org/ART1013test',
 'http://icondataset.org/ART1017test',
 'http://icondataset.org/ART1043',
 'http://icondataset.org/ART1066',
 'http://icondataset.org/ART1124',
 'http://icondataset.org/ART1126',
 'http://icondataset.org/ART1199',
 'http://icondataset.org/ART1266',
 'http://icondataset.org/ART1271',
 'http://icondataset.org/ART1273',
 'http://icondataset.org/ART1274',
 'http://icondataset.org/ART1283',
 'http://icondataset.org/ART1288',
 'http://icondataset.org/ART1338',
 'http://icondataset.org/ART1374',
 'http://icondataset.org/ART1422',
 'http://icondataset.org/ART1424',
 'http://icondataset.org/ART1467',
 'http://icondataset.org/ART1525'}

In [ ]:
tot_simb_art_pan = {<http://icondataset.org/ART1001test> <http://icondataset.org/ART1002test> <http://icondataset.org/ART1004> <http://icondataset.org/ART1009test> <http://icondataset.org/ART1011test> <http://icondataset.org/ART1013test> <http://icondataset.org/ART1017test> <http://icondataset.org/ART1043> <http://icondataset.org/ART1066> <http://icondataset.org/ART1124> <http://icondataset.org/ART1126> <http://icondataset.org/ART1199> <http://icondataset.org/ART1266> <http://icondataset.org/ART1271> <http://icondataset.org/ART1273> <http://icondataset.org/ART1274> <http://icondataset.org/ART1283> <http://icondataset.org/ART1288> <http://icondataset.org/ART1338> <http://icondataset.org/ART1374> <http://icondataset.org/ART1422> <http://icondataset.org/ART1424> <http://icondataset.org/ART1467> <http://icondataset.org/ART1525>}

In [ ]:
tot_com_pan_sim = {<http://icondataset.org/deer-souls> <http://icondataset.org/dove-holy-spirit> <http://icondataset.org/white-purity> <http://icondataset.org/cone-shell-fertility> <http://icondataset.org/bat-melancholy> <http://icondataset.org/dragon-biting-its-tail-time> <http://icondataset.org/blue-hue-of-heaven> <http://icondataset.org/dog-envy> <http://icondataset.org/skull-death-personified> <http://icondataset.org/dolphin-haste> <http://icondataset.org/four-wings-four-seasons> <http://icondataset.org/three-heads-three-forms-of-time-past-present-future> <http://icondataset.org/ox-christ> <http://icondataset.org/serpent-perfidy> <http://icondataset.org/myrtle-everlasting-love> <http://icondataset.org/ixion-lewdness> <http://icondataset.org/myrtle-love> <http://icondataset.org/lion-fury> <http://icondataset.org/wolf-greed> <http://icondataset.org/mermaid-persuasion> <http://icondataset.org/mirror-introspection> <http://icondataset.org/rose-virginity> <http://icondataset.org/ivy-eternal-love> <http://icondataset.org/tantalus-avarice> <http://icondataset.org/apollo-idolatry> <http://icondataset.org/bacchus-folly> <http://icondataset.org/bear-anger> <http://icondataset.org/bull-christ> <http://icondataset.org/dolphin-persuasion> <http://icondataset.org/fire-luxury> <http://icondataset.org/hercules-egypt> <http://icondataset.org/jupiter-pride> <http://icondataset.org/mars-discord> <http://icondataset.org/pluto-avarice> <http://icondataset.org/sisyphus-pride>}

In [185]:
# among the set of artTotPanSim, how many times does the symbol appear in an artwork? 
#totPanofskySimulation


cq3 = '''
prefix data: <http://icondataset.org/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
prefix sim: <https://w3id.org/simulation/ontology/>
prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
prefix crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?symbol (count(?artwork) as ?tot) WHERE {
VALUES ?symbol {<http://icondataset.org/deer-souls> <http://icondataset.org/dove-holy-spirit> <http://icondataset.org/white-purity> <http://icondataset.org/cone-shell-fertility> <http://icondataset.org/bat-melancholy> <http://icondataset.org/dragon-biting-its-tail-time> <http://icondataset.org/blue-hue-of-heaven> <http://icondataset.org/dog-envy> <http://icondataset.org/skull-death-personified> <http://icondataset.org/dolphin-haste> <http://icondataset.org/four-wings-four-seasons> <http://icondataset.org/three-heads-three-forms-of-time-past-present-future> <http://icondataset.org/ox-christ> <http://icondataset.org/serpent-perfidy> <http://icondataset.org/myrtle-everlasting-love> <http://icondataset.org/ixion-lewdness> <http://icondataset.org/myrtle-love> <http://icondataset.org/lion-fury> <http://icondataset.org/wolf-greed> <http://icondataset.org/mermaid-persuasion> <http://icondataset.org/mirror-introspection> <http://icondataset.org/rose-virginity> <http://icondataset.org/ivy-eternal-love> <http://icondataset.org/tantalus-avarice> <http://icondataset.org/apollo-idolatry> <http://icondataset.org/bacchus-folly> <http://icondataset.org/bear-anger> <http://icondataset.org/bull-christ> <http://icondataset.org/dolphin-persuasion> <http://icondataset.org/fire-luxury> <http://icondataset.org/hercules-egypt> <http://icondataset.org/jupiter-pride> <http://icondataset.org/mars-discord> <http://icondataset.org/pluto-avarice> <http://icondataset.org/sisyphus-pride>}
  ?image icon:hasSymbol ?symbol; 
      ^icon:recognizedImage / icon:aboutWorkOfArt ?artwork .
} GROUP BY ?symbol ?tot
ORDER BY DESC(?tot)

'''

cq3res = pang.query(cq3)

for row in cq3res:
    print(row[0], row[1])


http://icondataset.org/dragon-biting-its-tail-time 3
http://icondataset.org/dolphin-haste 2
http://icondataset.org/four-wings-four-seasons 2
http://icondataset.org/bear-anger 2
http://icondataset.org/deer-souls 1
http://icondataset.org/dove-holy-spirit 1
http://icondataset.org/white-purity 1
http://icondataset.org/cone-shell-fertility 1
http://icondataset.org/bat-melancholy 1
http://icondataset.org/blue-hue-of-heaven 1
http://icondataset.org/dog-envy 1
http://icondataset.org/skull-death-personified 1
http://icondataset.org/three-heads-three-forms-of-time-past-present-future 1
http://icondataset.org/ox-christ 1
http://icondataset.org/serpent-perfidy 1
http://icondataset.org/myrtle-everlasting-love 1
http://icondataset.org/ixion-lewdness 1
http://icondataset.org/myrtle-love 1
http://icondataset.org/lion-fury 1
http://icondataset.org/wolf-greed 1
http://icondataset.org/mermaid-persuasion 1
http://icondataset.org/mirror-introspection 1
http://icondataset.org/rose-virginity 1
http://icondat

# Synonyms through wordnet

In [106]:
from nltk.corpus import wordnet as wn

In [111]:
# code adapted from: https://www.holisticseo.digital/python-seo/nltk/wordnet
syn_dict = {}
def synonym_antonym_extractor(dictionary, syn_dict):
    syn_names = {}
    names_list = list(dictionary.keys())
    for n in names_list: 
    # from nltk.corpus import wordnet
        synonyms = []
        syn = wn.synsets(n) # it does already a world lookup, synonyms included
        if syn != []: 
            for l in syn[0].lemmas():
                synonyms.append(l.name())
               # print(syn[0].name(),"-definition:", syn[0].definition(),"-synonyms:", l.name())
            syn_dict[n] = synonyms 
            wname = syn[0].name().split(".")[0]
            if fuzz_ratio_mean(n, wname) > 80 and n in synonyms: 
                if len(synonyms) > 1:
                    syn_names[n] = synonyms # wname
        
            print(n, syn[0].name(),"-definition:", syn[0].definition(),"-synonyms:", synonyms)
    # print(set(antonyms))
    return syn_names
RcPanofskySynonyms = synonym_antonym_extractor(panofsky, syn_dict)

idolatry idolatry.n.01 -definition: religious zeal; the willingness to serve God -synonyms: ['idolatry', 'devotion', 'veneration', 'cultism']
folly folly.n.01 -definition: the trait of acting stupidly or rashly -synonyms: ['folly', 'foolishness', 'unwiseness']
melancholy melancholy.n.01 -definition: a feeling of thoughtful sadness -synonyms: ['melancholy']
anger anger.n.01 -definition: a strong emotion; a feeling that is oriented toward some real or supposed grievance -synonyms: ['anger', 'choler', 'ire']
fertilising fertilize.v.02 -definition: make fertile or productive -synonyms: ['fertilize', 'fecundate', 'fertilise']
christ jesus.n.01 -definition: a teacher and prophet born in Bethlehem and active in Nazareth; his life and sermons form the basis for Christianity (circa 4 BC - AD 29) -synonyms: ['Jesus', 'Jesus_of_Nazareth', 'the_Nazarene', 'Jesus_Christ', 'Christ', 'Savior', 'Saviour', 'Good_Shepherd', 'Redeemer', 'Deliverer']
unity integrity.n.01 -definition: an undivided or unbro

In [112]:
RcPanofskySynonyms

{'idolatry': ['idolatry', 'devotion', 'veneration', 'cultism'],
 'folly': ['folly', 'foolishness', 'unwiseness'],
 'anger': ['anger', 'choler', 'ire'],
 'panic': ['panic', 'terror', 'affright'],
 'envy': ['envy', 'enviousness'],
 'haste': ['haste', 'hastiness', 'hurry', 'hurriedness', 'precipitation'],
 'persuasion': ['persuasion', 'suasion'],
 'time': ['time', 'clip'],
 'decade': ['decade', 'decennary', 'decennium'],
 'reason': ['reason', 'ground'],
 'pride': ['pride', 'pridefulness'],
 'purity': ['purity', 'pureness'],
 'fury': ['fury', 'rage', 'madness'],
 'discord': ['discord', 'strife'],
 'introspection': ['introspection', 'self-contemplation', 'self-examination'],
 'avarice': ['avarice', 'greed', 'covetousness', 'rapacity', 'avaritia'],
 'perfidy': ['perfidy', 'perfidiousness', 'treachery']}

In [ ]:
for tup in panofsky.items():
        for item in Simulacrum2Dict.items(): 
            for t in tup[1]:
                if t == item[0]: 
                   # print("panofsky simulacrum", t, "for the rc", tup[0])
                    for i in item[1]:
                     #   print("oderr simulacrum", i, "for the rc", item[0])
                        mean = fuzz_ratio_mean(tup[0], i)
                        if mean > 70: 
                           # print("panofsky simulacrum", t, "for the rc", tup[0])
                           # print("oderr simulacrum", i, "for the rc", item[0])
                            print("matched reality counterpart:", tup[0], " - ", i, "- for the simulacrum:", t, " - ", item[0])

In [31]:
cq1 = '''
PREFIX kb: <https://w3id.org/simulation/data/> 
PREFIX ns1: <http://www.w3.org/ns/prov-o-inverses#> 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX prov: <http://www.w3.org/ns/prov#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX sim: <https://w3id.org/simulation/ontology/> 
PREFIX wn: <http://wordnet-rdf.princeton.edu/lemma/> 

SELECT DISTINCT ?symulation ?simulacrum WHERE {
?symulation sim:hasRealityCounterpart kb:ornamentOfTheBridalChamber;
    sim:hasSimulacrum ?simulacrum .


}

'''

cq1res = newg.query(cq1)

for row in cq1res:
    print(row[0])

https://w3id.org/simulation/data/citron-ornamentOfTheBridalChamber
